In [ ]:
from env import *
from swayam import *

### Tool Calling with Output Hooks

In tool creation you can reuse structures using structure hooks discussed earlier.

In [ ]:
router = Router(run_id="test")

### Some Dummy Functions

Let's create some dummy functions

In [ ]:
from enum import Enum
class UnitEnum(Enum):
    CELSIUS = "celsius"
    FAHRENHEIT = "fahrenheit"

import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit):
    import random
    i = random.randint(-10, 40)
    # As the UnitEnum in externalised Structure is a different enum class, modifying the logic
    if not isinstance(unit, UnitEnum):
        unit = UnitEnum[unit.name]

    if unit == UnitEnum.CELSIUS:
        return f"{i}°C"
    elif unit == UnitEnum.FAHRENHEIT:
        return f"{i*9/5 + 32}°F"
    else:
        raise ValueError("Invalid unit")
    
def calculate(expression):
    return eval(expression)

def search_email(text):
    import re
    emails = re.findall(r'([\w\.-]+@[\w\.-]+)', text)
    if emails:
        return emails
    else:
        return None

In [ ]:
from swayam import Tool

WeatherInfo = Tool.build("WeatherInfo", 
                         target=get_current_weather, 
                         desc="Get the current weather in a given location",
                         call_structure=Structure.WeatherInfo
)

Calculator = Tool.build("Calculator",
                        target=calculate,
                        desc="Evaluates a mathematical expression",
                        call_structure=Structure.Calculation
)

EmailSearch = Tool.build("EmailSearch",
                        target=search_email,
                        desc="Searches email addresses in a given text",
                        call_structure=Structure.EmailSearch
)

In [ ]:
tools = [WeatherInfo, Calculator, EmailSearch]

In [ ]:
user_prompt = Prompt.text("What's the weather like in Boston?", tools=tools)
router.execute(user_prompt)

In [ ]:
router.execute(Prompt.text("What is 7 *30 + 13?", tools=tools))

In [ ]:
text = "This is a sample text with a@b.com and c@d.com addresses"
prompt_text = f"""Find whether there is an email address in the following text marked with triple backticks 
            ```{text}```
            """
router.execute(Prompt.text(prompt_text, tools=tools))